### Schema Evolution Mode 
#### None , fail on new column, Rescue,Add new column(defualt)

In [0]:
landing_zone = '/Volumes/workspace/default/lateflow_connect_volume'
orders_data = landing_zone + '/orders_data'
checkpoint_path = landing_zone + '/manuallyInferschemacheckpoint'

### schemaEvolutionMode = none.> it will ignore the file , rescue data will not there 

In [0]:
orders_df = (spark.readStream
.format("cloudFiles")
.option("cloudFiles.format", "csv")
.option("cloudFiles.schemaHints", "order_id bigint, order_date string")
.option("cloudFiles.inferSchema", "true")
.option("cloudFiles.inferColumnTypes", "true")
.option("cloudFiles.schemaEvolutionMode", "None")
.option("cloudFiles.schemaLocation", checkpoint_path)
.load(orders_data))

In [0]:
orders_df.printSchema();

In [0]:
%sql
drop table if exists ordersdeltastream;
create table ordersdeltastream;

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))

In [0]:
%sql
select * from ordersdeltastream;

### add new file which has new column, it will ignore the new column
### for mismatch data type , it will have null value. it will be data loss 
### no _rescue_data column

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))

In [0]:
%sql
select * from ordersdeltastream;

### Fail on New column schema evolution mode

In [0]:
orders_df = (spark.readStream
.format("cloudFiles")
.option("cloudFiles.format", "csv")
.option("cloudFiles.schemaHints", "order_id bigint, order_date string")
.option("cloudFiles.inferSchema", "true")
.option("cloudFiles.inferColumnTypes", "true")
.option("cloudFiles.schemaEvolutionMode", "failOnNewColumns")
.option("cloudFiles.schemaLocation", checkpoint_path)
.load(orders_data))

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))

In [0]:
%sql
select * from ordersdeltastream;


### add file with new column , it will fail. retry will not work

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))

### SchemaEvolutionMode = rescue
#### schema never evolve, data will be added in _rescued_data column

In [0]:
orders_df = (spark.readStream
.format("cloudFiles")
.option("cloudFiles.format", "csv")
.option("cloudFiles.schemaHints", "order_id bigint, order_date string")
.option("cloudFiles.inferSchema", "true")
.option("cloudFiles.inferColumnTypes", "true")
.option("cloudFiles.schemaEvolutionMode", "rescue")
.option("cloudFiles.schemaLocation", checkpoint_path)
.load(orders_data))

In [0]:
%sql
drop table if exists ordersdeltastream;
create table ordersdeltastream;

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))

In [0]:
%sql
select * from ordersdeltastream;
    


### add a file with new column. it will get added in rescued_data column

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))

In [0]:
%sql
select * from ordersdeltastream;

### schemaEvolutionMode : addNewColumns 
#### schema will be evolved when extra column detected in the source data.